In [ ]:
! pip install imutils

import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import imutils
import pathlib
import time
import PIL as pil
import shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import gc
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import joblib



IMAGE_SIZE = (224, 224)
BASE_LR = 2e-5
EPOCH = 20
BATCH_SIZE = 32

In [ ]:
def crop_img(img, image_size=(224, 224)):
    """
    Finds the extreme points on the image and crops the rectangular region
    Resizes to the target image size using bicubic interpolation
    """
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.GaussianBlur(gray, (3, 3), 0)

    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if len(cnts) == 0:
        print("Warning: No contours found, returning resized original image.")
        return cv2.resize(img, image_size, interpolation=cv2.INTER_CUBIC)
    
    c = max(cnts, key=cv2.contourArea)

    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    ADD_PIXELS = 0  # Can be tuned for padding
    new_img = img[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()
    
    # Resize using bicubic interpolation
    new_img = cv2.resize(new_img, image_size, interpolation=cv2.INTER_CUBIC)
    
    return new_img

In [ ]:
def image_preprocessing(source_dir, saved_root_dir, image_size=(224, 224), channels=3):
    if not os.path.exists(source_dir):
        raise Exception(f"Source directory: {source_dir} does not exist")
    if not os.path.isdir(source_dir):
        raise Exception(f"Source path: {source_dir} is not a directory")

    if not os.path.exists(saved_root_dir):
        os.makedirs(saved_root_dir)
        
    source_dir_path = pathlib.Path(source_dir)
    
    for p in tqdm(source_dir_path.iterdir(), desc="Processing folders"):
        dir_name = str(p).split("/")[-1]
        for fp in p.iterdir():
            filename = str(fp).split("/")[-1]

            img = tf.io.read_file(str(fp))
            img = tf.image.decode_jpeg(img, channels=channels)
            img = crop_img(img.numpy(), image_size)
            img = pil.Image.fromarray(img)

            saved_dist_dir = os.path.join(saved_root_dir, dir_name)
            if not os.path.exists(saved_dist_dir):
                os.makedirs(saved_dist_dir)

            img_dist_path = os.path.join(saved_dist_dir, filename)
            img.save(img_dist_path)
    print(f"\n✅ All images processed and saved to: {saved_root_dir}")


In [ ]:
image_preprocessing("/kaggle/input/brain-tumor-mri-dataset/Training",
                   "/kaggle/working/brain-tumor-mri-dataset/Training",
                   image_size=IMAGE_SIZE)

image_preprocessing("/kaggle/input/brain-tumor-mri-dataset/Testing",
                   "/kaggle/working/brain-tumor-mri-dataset/Testing",
                   image_size=IMAGE_SIZE)

# Datasets

In [ ]:
root_dir_path = "/kaggle/working/brain-tumor-mri-dataset"

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/working/brain-tumor-mri-dataset/Training",
    label_mode="categorical",
    batch_size=32,
    image_size=IMAGE_SIZE,
    seed=42
)


test_ds = tf.keras.utils.image_dataset_from_directory(
    "/kaggle/input/brain-tumor-mri-dataset/Testing",  # original, not preprocessed
    label_mode="categorical",
    image_size=IMAGE_SIZE,
    batch_size=32,
    shuffle=False  # Important for consistent evaluation
)

In [ ]:
print(train_ds.class_names)
print(test_ds.class_names)

cls_to_id = {c:i for i, c in enumerate(train_ds.class_names)}
print(cls_to_id)
id_to_cls = {i:c for i, c in enumerate(train_ds.class_names)}
print(id_to_cls)

In [ ]:
with open("class_to_id.txt", "w") as f:
    for k, v in cls_to_id.items():
        f.write(f"{k}\t{v}\n")

with open("id_to_class.txt", "w") as f:
    for k, v in id_to_cls.items():
        f.write(f"{k}\t{v}\n")

In [ ]:
with open("class_to_id.txt", "r") as f:
    for line in f.readlines():
        cls, label = line.replace("\n","").split("\t")
        print(cls, int(label))
print("\n")
with open("id_to_class.txt", "r") as f:
    for line in f.readlines():
        label, cls = line.replace("\n","").split("\t")
        print(int(label), cls)

In [ ]:
def class_weight_from_one_hot(ds):
    class_labels = []
    if ds.__class__.__name__ == "_BatchDataset":
        ds = ds.unbatch()
    
    for _, onehot in ds:
        class_labels.append(tf.argmax(onehot).numpy())
    
    unique_classes = np.unique(class_labels)
    class_weights = compute_class_weight(class_weight="balanced", 
                                         classes=unique_classes,
                                         y=class_labels)
    return {i:w for i, w in enumerate(class_weights)}

    
class_weights = class_weight_from_one_hot(train_ds)
print(class_weights)

In [ ]:
for images, labels in train_ds.take(1):
    image, label = images[0], labels[0]
    plt.figure()
    plt.imshow(tf.cast(image, tf.uint8))
    plt.title(train_ds.class_names[tf.argmax(label).numpy()])
    plt.show()

In [ ]:
class GAM(tf.keras.layers.Layer):
    def __init__(self, reduction_ratio=16, **kwargs):
        super(GAM, self).__init__(**kwargs)
        self.reduction_ratio = reduction_ratio

    def build(self, input_shape):
        channels = input_shape[-1]
        self.channel_mlp = tf.keras.Sequential([
            tf.keras.layers.Dense(channels // self.reduction_ratio, activation='relu'),
            tf.keras.layers.Dense(channels)
        ])
        self.spatial_conv = tf.keras.Sequential([
            tf.keras.layers.Conv2D(channels // self.reduction_ratio, 7, padding='same', activation='relu'),
            tf.keras.layers.Conv2D(1, 7, padding='same', activation='sigmoid')
        ])

    def call(self, inputs):
        # Channel attention
        channel_att = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)
        channel_att = self.channel_mlp(channel_att)

        # Spatial attention
        spatial_att = self.spatial_conv(inputs)

        return inputs * channel_att * spatial_att

class ECA(tf.keras.layers.Layer):
    def __init__(self, gamma=2, b=1, **kwargs):
        super(ECA, self).__init__(**kwargs)
        self.gamma = gamma
        self.b = b

    def build(self, input_shape):
        channels = input_shape[-1]
        self.kernel_size = int(abs((tf.math.log(tf.cast(channels, tf.float32), 2) + self.b) / self.gamma))
        self.kernel_size = self.kernel_size if self.kernel_size % 2 else self.kernel_size + 1
        self.conv = tf.keras.layers.Conv1D(1, kernel_size=self.kernel_size, padding='same', use_bias=False)

    def call(self, inputs):
        # Global Average Pooling (Keep Channel Dimension)
        x = tf.reduce_mean(inputs, axis=[1, 2], keepdims=True)

        # Reshape for Conv1D
        x = tf.squeeze(x, axis=[1, 2])
        x = tf.expand_dims(x, axis=-1)

        # Apply 1D Convolution for Channel Attention
        x = self.conv(x)
        x = tf.sigmoid(x)

        # Reshape to match original input dimensions
        x = tf.reshape(x, [-1, 1, 1, inputs.shape[-1]])

        return inputs * x


# Model 

EfficientNetV2

In [ ]:
from tensorflow.keras.models import load_model

# Define or import GAM and ECA before loading
# from your_custom_layers import GAM, ECA

model = load_model(
    '/kaggle/input/keras_m/tensorflow2/default/1/brain_tumor_detector.keras',
    custom_objects={'GAM': GAM, 'ECA': ECA}
)

for layer in model.layers:
    print(layer.name)


In [ ]:
import os
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import random
from IPython.display import Image
# import imutils   


import keras
import tensorflow.keras as K

import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, ImageDataGenerator, array_to_img, img_to_array
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
has_conv_layers = any(isinstance(layer, K.layers.Conv2D) for layer in model.layers)
print(f"Model has convolutional layers: {has_conv_layers}")


In [ ]:
def VizGradCAM(model, image, interpolant=0.5, plot_results=True):
    """Fixed Grad-CAM implementation for your model"""
    assert (interpolant > 0 and interpolant < 1), "Heatmap Interpolation Must Be Between 0 - 1"
    
    try:
        # STEP 1: Preprocess image and make prediction
        original_img = np.asarray(image, dtype=np.float32)
        img = np.expand_dims(original_img, axis=0)
        prediction = model.predict(img, verbose=0)
        prediction_idx = np.argmax(prediction)

        # STEP 2: Get the ECA layer output (last conv-like layer)
        eca_layer = model.get_layer('eca_5')
        
        # Create gradient model
        grad_model = Model(
            inputs=model.inputs,
            outputs=[eca_layer.output, model.output]
        )
        
        # STEP 3: Compute gradients
        with tf.GradientTape() as tape:
            conv_output, preds = grad_model(img, training=False)
            loss = preds[:, prediction_idx]

        # Compute gradients
        grads = tape.gradient(loss, conv_output)[0]  # Take first (and only) batch element
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1))  # Average over spatial dimensions
        
        # Weight the feature maps by gradient importance
        conv_output = conv_output[0].numpy()  # Shape: (8, 8, 1280)
        pooled_grads = pooled_grads.numpy()   # Shape: (1280,)
        
        # Multiply each feature map by its corresponding gradient mean
        for i in range(pooled_grads.shape[0]):
            conv_output[:, :, i] *= pooled_grads[i]
        
        # Create heatmap by averaging across channels
        heatmap = np.mean(conv_output, axis=-1)
        
        # Post-process heatmap
        heatmap = np.maximum(heatmap, 0)
        heatmap /= np.max(heatmap)  # Normalize between 0-1
        heatmap = cv2.resize(heatmap, (original_img.shape[1], original_img.shape[0]))
        heatmap = np.uint8(255 * heatmap)

        # Apply colormap
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        
        # Prepare original image
        original_img_uint8 = np.uint8(255 * (original_img - original_img.min()) / 
                            (original_img.max() - original_img.min()))
        
        # Superimpose
        superimposed_img = cv2.addWeighted(original_img_uint8, interpolant, 
                                         heatmap, 1 - interpolant, 0)
        
        # Display
        plt.figure(figsize=(10, 5))
        
        plt.subplot(1, 2, 1)
        plt.imshow(original_img_uint8)
        plt.title("Original Image")
        plt.axis('off')
        
        plt.subplot(1, 2, 2)
        plt.imshow(superimposed_img)
        plt.title("Grad-CAM")
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"Error in VizGradCAM: {str(e)}")
        import traceback
        traceback.print_exc()
        if not plot_results:
            return None

In [ ]:
# Load your image (without augmentation)
def load_image(path):
    img = tf.keras.preprocessing.image.load_img(path, target_size=IMAGE_SIZE)
    img = tf.keras.preprocessing.image.img_to_array(img)
    return img

# Apply Grad-CAM
mri_image = load_image("/kaggle/input/brain-tumor-mri-dataset/Testing/glioma/Te-glTr_0000.jpg")
VizGradCAM(model, mri_image)

In [ ]:
# Apply Grad-CAM
mri_image = load_image("/kaggle/input/brain-tumor-mri-dataset/Testing/meningioma/Te-meTr_0001.jpg")
VizGradCAM(model, mri_image)

In [ ]:
# Apply Grad-CAM
mri_image = load_image("/kaggle/input/brain-tumor-mri-dataset/Testing/notumor/Te-noTr_0006.jpg")
VizGradCAM(model, mri_image)

In [ ]:
# Apply Grad-CAM
mri_image = load_image("/kaggle/input/brain-tumor-mri-dataset/Testing/pituitary/Te-piTr_0001.jpg")
VizGradCAM(model, mri_image)